In [47]:
import pandas as pd
model = 'model_11111'

import yaml
import os

with open(f'src/config/params/{model}.yaml', 'r') as file:
    config = yaml.safe_load(file)
print(config)

{'name': 'model_11111', 'data_path': 'data/ELISA_merged.csv', 'attributes': ['Acceleration', 'CO2', 'Charginginfrastructure', 'Chargingspeed', 'Homechargeravailability', 'Homedistnearestcharger', 'Operationcost', 'Purchaseprice', 'Range', 'Size', 'Yearlycost'], 'characteristics': ['age', 'gender', 'income', 'education', 'occupation'], 'scenario': {'Purchaseprice': -0.1}, 'nested': True, 'estimation': False}


In [48]:
from models import Data
name = config['name']
data_path = config['data_path']
attributes = config['attributes']
characteritics = config['characteristics']
scenario = config['scenario']
nested = config['nested']
database = Data(data_path, attributes, characteritics, scenario)

d:\Alexandre\Documents\Cours\DTU\Quantitative modelling of behaviour\Assigment\code2\models\database.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data[raw_data['ICV'] == 1][key] = (1+value) * raw_data[key]
d:\Alexandre\Documents\Cours\DTU\Quantitative modelling of behaviour\Assigment\code2\models\database.py:106: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  wide_data[x+'_'+str(i)] = self.pivot_data[(x, i)]


In [49]:
y = (database.get_testing_set()['CHOICE'].value_counts()/database.get_testing_set()['CHOICE'].value_counts().sum()).sort_index()
y

1     0.068762
2     0.089542
3     0.151039
4     0.069944
5     0.039872
6     0.010644
7     0.054232
8     0.066734
9     0.168272
10    0.088360
11    0.030242
12    0.003210
13    0.024497
14    0.027032
15    0.072985
16    0.026187
17    0.006420
18    0.002027
Name: CHOICE, dtype: float64

In [50]:
val_df = pd.read_csv(f"results/validations/{name}_{nested}.csv")
val_df = val_df.drop('Unnamed: 0', axis=1)
val_df.index = val_df.index + 1
val_df['true'] = y
val_df

,alternative,true,pred
1,ICV_MINI,0.068762,0.063943
2,ICV_LIL,0.089542,0.093146
3,ICV_MEL,0.151039,0.153493
4,ICV_STOR,0.069944,0.067031
5,ICV_PREM,0.039872,0.036197
6,ICV_LUK,0.010644,0.007685
7,BEV_MINI,0.054232,0.052233
8,BEV_LIL,0.066734,0.067013
9,BEV_MEL,0.168272,0.174713
10,BEV_STOR,0.088360,0.091114


In [51]:
scenario_df = pd.read_csv(f"results/scenarios/{name}_{nested}_{next(iter(scenario))}.csv")
scenario_df.index = scenario_df.index + 1
scenario_df = scenario_df.drop(['Unnamed: 0', 'true'], axis=1)
try: 
    scenario_df = scenario_df.rename({'pred': 'forecast'}, axis=1)
except KeyError:
    pass
val_df = val_df.merge(scenario_df, on='alternative')
val_df

,alternative,true,pred,forecast
0,ICV_MINI,0.068762,0.063943,0.061279
1,ICV_LIL,0.089542,0.093146,0.089985
2,ICV_MEL,0.151039,0.153493,0.146228
3,ICV_STOR,0.069944,0.067031,0.062837
4,ICV_PREM,0.039872,0.036197,0.033456
5,ICV_LUK,0.010644,0.007685,0.007176
6,BEV_MINI,0.054232,0.052233,0.051434
7,BEV_LIL,0.066734,0.067013,0.068650
8,BEV_MEL,0.168272,0.174713,0.180498
9,BEV_STOR,0.088360,0.091114,0.095000


In [52]:
import numpy as np
rmse = np.sqrt(((val_df['pred'] - val_df['true'])**2).sum()/val_df.shape[0])
print(rmse)

0.002795555445660191


In [53]:
val_df['tech'] = [t.split("_",)[0] for t in val_df['alternative']]
validation_tech = val_df.groupby('tech').sum()
rmse = np.sqrt(((validation_tech['pred'] - validation_tech['true'])**2).sum()/validation_tech.shape[0])
print(rmse)
print(validation_tech)

0.006773410887366571
          true      pred  forecast
tech                              
BEV   0.411049  0.419333  0.434074
ICV   0.429802  0.421495  0.400960
PHEV  0.159149  0.159172  0.164966


In [54]:
import pickle
parameters = pickle.load(open(f"results/estimations/{name}_{nested}.pickle", "rb" ))
dict = {}
for beta in parameters.betas:
    dict[f"{beta.name}"] =  beta.value
next(iter(dict))

'ASC_BEV_LIL'